# Export Existing Azure SQL Server Resources
Export notebook that will utilize the ADP resources

## Notebook Variables
| Line | Variable | Description |
| -- | -- | -- |
| 1 | AdpSubscription | Azure Subscription ID/Name for the ADP Resource Group # Both RG are assumed to be in the same subscription |
| 2 | AdpResourceGroup | Azure Resource Group which contains the ADP Resources | 
| 3 | SourceResourceGroup | Azure ResourceGroup where the sql server to be exported exists | 
| 4 | LogicalSQLServerName | Logical sql server name of the sql server to be exported | 
| 5 | StorageAccount | target storage account to store exported files # any storage account, but must be in the same RG as the ADP resources | 
| 6 | AdpFunc | |
| 7 | AdpBatch | | 
| 8 | AdpVNET | | 

In [ ]:
$AdpSubscription    = ""
$AdpResourceGroup   = ""
$SourceResourceGroup= ""
$LogicalSQLServer   = ""
$StorageAccount     = ""
$AdpFunc            = $AdpResourceGroup + "Control"
$AdpBatch           = $AdpResourceGroup.ToLower() + "batch"
$AdpVNET            = $AdpResourceGroup + "Vnet"

## Steps
Gather input:
* [ ] Connect to Azure Subscription
* [ ] Choose Resource Group (Read access required)
* [ ] Choose Microsoft SQL Server Resources to export
* [ ] Choose/Create Migration Storage 
* [ ] Install Application + Data Portability function (orchestrator service)
* [ ] Install ADP Azure Batch processing pipeline
* [ ] Store SqlPackage.exe in Migration Storage for orchestrator to hand to Az Batch

Execute:
* [ ] Check all pre-requisites
* [ ] Kick off orchestrator service

Monitor:
* [ ] Check export status.

## Notebook Functions
Defines logical functions for the rest of the notebook. Function blocks are combined in a single cell that can be collapsed for readability or expanded for further examination. Nothing is executed until called later in the notebook. As a result, this cell is a requirement for any of the other cells below it. 

In [ ]:
# Expand cell to view framework

function Login-Azure
{    
    # query azure locations to test for existing az login session exists with valid access tocken
    $azureLocations = az account list-locations -o JSON 2>$null | ConvertFrom-Json
    
    if (!$azureLocations){ #If there are no az locations, there is no existing az login session
        $subscriptions = az login -o JSON | ConvertFrom-Json      # Login   
    }
    else {
        $subscriptions = az account list -o JSON | ConvertFrom-Json # getting subscriptions for the user to use in gridview
    }

    if(![string]::IsNullOrWhiteSpace($AdpSubscription)) #If there is a subscription specified by user in the variables section
    {
        $specified_Subscription= az account show --subscription $AdpSubscription -o json |ConvertFrom-Json 
        if (!$specified_Subscription) #if specified subscription is not valid
        {       
            $currentUser= az ad signed-in-user show --query "{displayName:displayName,UPN:userPrincipalName}" -o json|ConvertFrom-Json # get current logged in user infomration
            Write-Host "Refer below for the list of subscriptions for logged in account '$($currentUser.UPN)'`n"
            az account list --query "[].{Name:name,SubscriptionID:id}" -o table          # list subscriptions under current logged in account
        }
        else { # if specified subscription is valid
            Write-Output "Using subscription... '$($specified_Subscription.name)' ... '$($specified_Subscription.id)'" 
        }
    }
    else { # if no subscription is specified, users are given a gridview to select subscription from

        $selectedSubscription = $subscriptions | Select-Object -Property Name, Id | Out-GridView -PassThru
        $SubscriptionId = $selectedSubscription.Id
        $Subscription = $selectedSubscription.Name   
        $AdpSubscription = $subscription  
        Write-Output "Using subscription... '$AdpSubscription' ... '$SubscriptionId'"    
    }    
}

function Verify-ADPResources
{    
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ADPResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$FunctionName,  
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BatchAccountName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$VNetName,
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$ApplicationName="SqlPackageWrapper",  
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$ApplicationPackageVersionName="1",
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$SubNetName="default" 
    )    

# validate Subscription
$specified_Subscription= az account show --subscription $Subscription -o json | ConvertFrom-Json
if(!$specified_Subscription){
    $currentUser= az ad signed-in-user show --query "{displayName:displayName,UPN:userPrincipalName}" -o json|ConvertFrom-Json # get current logged in user information
    Write-Host "Refer below for the list of subscriptions for logged in account '$($currentUser.UPN)'`n"
    az account list --query "[].{Name:name,SubscriptionID:id}" -o table          # list subscriptions under current logged in account
    return } 
# validate ResourceGroup    
$specified_ResourceGroup= az group show -n $ADPResourceGroupName --subscription $Subscription -o json | ConvertFrom-Json
if(!$specified_ResourceGroup) {        
        return
        } 

$Installed = [ordered]@{} # ordered hash to store status of installation
$countError=0

#Verify if VNet exists       
$specified_VNet= az network vnet show -n $VNetName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null  |ConvertFrom-Json   
if(!$specified_VNet) {
    $Installed['VNET']="Not Found"
    $countError++
}   
else { 
    $existingVnetSubnet =  az network vnet subnet show -n $SubNetName --vnet-name $VNetName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null |ConvertFrom-Json
    if(!$existingVnetSubnet){
        $Installed['VNET']="Default Subnet under"+ $VNetName + "Not Found"
        $countError++
        }
    else {
    $Installed['VNET']="Installed"
    }
 }

#Verify if FunctionApp Exists
$specified_FunctionApp = az functionapp show -n $FunctionName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
if(!$specified_FunctionApp)
{
    $Installed['FunctionApp']="Not Installed"
    $countError++
}
else
{
    $Installed['FunctionApp']="Installed"
} 

#check if Batch account exists
$specified_BatchAccount = az batch account show -n $BatchAccountName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
if(!$specified_BatchAccount)
{
    $Installed['Batch']="Not Installed"
    $countError++
}
else
{
    $appPackageInstalled = az batch application package show --application-name $ApplicationName --version-name $ApplicationPackageVersionName -n $BatchAccountName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
    $connectedToStorage= $specified_BatchAccount.autoStorage 
    if($connectedToStorage -and $appPackageInstalled){ # BatchAccount connected to storageaccount and  applicationpackage is installed
            $Installed['Batch']="Installed"
            $Installed['Batch_ApplicationPackage']="Installed"
            $Installed['Batch_StorageAccount']="Connected to storage- "+$connectedToStorage.storageAccountId.Split("/")[-1]
        }
    if(!$connectedToStorage)
    {
        $Installed['Batch_StorageAccount']='Not Found'
        $countError++
    }  
    if(!$appPackageInstalled)
    {
        $Installed['Batch_ApplicationPackage']="Not Found"
        $countError++
    }    
}

if ($countError -gt 0){
    Write-Output "ADP Resources are not installed correctly. Please refer the list below and use the Bootstrap NB to install ADP Resources"
}
$Installed
if ($countError -eq 0){
    Write-Output "`nFound all ADP Resources."
}
}

function Prepare-InputForExportFunction
{    
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ADPResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$FunctionName,  
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BatchAccountName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$VNetName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$SourceRGName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$SqlServerName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$StorageAccountName
    )
    
    $InputResult = @{}
    # Build Header 
    ## get Function key
    $FunctionAppID =az functionapp show -n $FunctionName -g $ADPResourceGroupName --subscription $Subscription --query "[id]" -o JSON 2>$null | ConvertFrom-Json
    $DefaultHostKey = az rest --method post --uri "$FunctionAppID/host/default/listKeys?api-version=2018-11-01" --query "[functionKeys.default]" -o JSON 2>$null | ConvertFrom-Json
    ## Build Json Object for Headers
    $headers = @{
        'x-functions-key' = $DefaultHostKey
    }
    $InputResult['Header']=$headers

    # Build string for Function URL 
    $specified_Subscription= az account show --subscription $Subscription -o json |ConvertFrom-Json #Get SpecifiedSubscriptionID
    $SubscriptionID= $specified_Subscription.id
    $FunctionUrl = 'https://'+ $FunctionName +'.azurewebsites.net/api/subscriptions/'+ $SubscriptionID +'/resourceGroups/' + $ADPResourceGroupName + '/Export'
    $InputResult['FunctionURL']=$FunctionUrl

    # Set parameter variables for Body
    ## Get BatchAccountURL 
    $specified_Batch = az batch account show -n $BatchAccountName -g $ADPResourceGroupName  --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
    $BatchAccountURL = 'https://' + $specified_Batch.accountEndpoint      

    ## Get default SubNet ID for specified VNet
    $specified_VNet_SubNet = az network vnet subnet show -g $ADPResourceGroupName --vnet-name $VNetName -n 'default' --subscription $Subscription -o JSON |ConvertFrom-Json
    $VNetSubNetID = $specified_VNet_SubNet.id

    ## Create access token to source sql server
    $sourceAccessToken = az account get-access-token --resource=https://database.windows.net --query accessToken
    $sourceAccessToken

    ## Build JSon object for Body
    $Body = @{
        batchAccountUrl = $BatchAccountURL
        storageAccountName = $StorageAccountName  # any storage account, not neccessarily the one connected to the batch account
        sourceSqlServerResourceGroupName = $SourceRGName 
        sourceSqlServerName = $SQLServerName 
        accessToken= $sourceAccessToken
        VNetSubnetId= $VNetSubNetID
    }
    $json = $Body | ConvertTo-Json
    $InputResult['Body']=$json

    $InputResult
}

function Provision-FuncRBAC {
    [CmdletBinding()]
    param (
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$Subscription,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$ResourceGroupName,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$FunctionName,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$ScopeRGName,
        [Parameter(Mandatory=$false)][ValidateNotNullOrEmpty()][string]$Role="Contributor"
    )

    # Get the scope resource group's ID
    $scopeID = az group show --resource-group $ScopeRGName --subscription $Subscription --query "[id]" -o JSON | ConvertFrom-Json 
    if(!$scopeID) {
        Write-Output "Provision-FuncRBAC failed." 
        return }
    else { Write-Output "Found scope '$ScopeRGName' with ID... '$scopeID'"
    }

    # Get the az function principal id
    $app_PrincipalID = az functionapp show -n $FunctionName --resource-group $ResourceGroupName --subscription $Subscription --query "[identity.principalId]" -o JSON  | ConvertFrom-Json  
    if(!$app_PrincipalID) {
        Write-Output "Provision-FuncRBAC failed." 
        return }
    else {  Write-Output "Found principal id of Azure function '$FunctionName'... '$app_PrincipalID'"
    }

    # Verify if a role assignment has been created for function
    $app_RoleAssignmentDefinition= az role assignment list --subscription $Subscription --assignee $app_PrincipalID --scope $scopeID --query "[].roleDefinitionName" -o JSON 2>$null  | ConvertFrom-Json

    if($app_RoleAssignmentDefinition -eq $Role)
    {
        Write-Output "Found Role Assignment for Principal ID.. '$app_PrincipalID' with Role.. '$app_RoleAssignmentDefinition' . No work needed"
    }
    else
    {
        # Continue to setup RBAC, once we verify an assignment is not setup and all the resources exist
        Write-Output "Creating new role assignment by running: 'az functionapp identity assign -n $FunctionName --role $Role -g $ResourceGroupName --scope $scopeID --subscription $Subscription'"
        Write-Warning "If your account does not have the access to assign new roles as Owner or User Access Administrator for the resource group, than you will need to contact your Azure AD Administrator to assign a service principle using the commands above"
        az functionapp identity assign -n $FunctionName --role $Role -g $ResourceGroupName --scope $scopeID --subscription $Subscription 
    }
}
Write-Host "Helper Functions Created successfully"  

## Connect to Azure Account
Run the below cell to login to an Azure account. <b>Be sure to check the Windows Taskbar for a login dialog box underneath the notebook or other windows or by pressing Alt+TAB.</b>

In [ ]:
Login-Azure 

## Verify ADP Resources 
Verify if ADP resources exists in specified Resource Group

In [ ]:
Verify-ADPResources -Subscription $AdpSubscription -ADPResourceGroupName $AdpResourceGroup `
                     -BatchAccountName $AdpBatch -FunctionName $AdpFunc -VNetName $AdpVNET 

## Verify RBAC of Azure Function
Roles based access control is a function of Azure that assigns services to a role with a specific access scope (or area of access). The ADP Orchestrator function requires Contributor access over the Resource Group where the SQL Server to be exported exists. The function below will attempt to create the role assignment. Any user executing this notebook will need to have Owner or User Access Administrator permissions to the Resource Group to assign the permission. Otherwise, contact your Azure AD Administrator. 

In [ ]:
Provision-FuncRBAC -FunctionName $AdpFunc -ScopeRGName $SourceResourceGroup -ResourceGroupName $AdpResourceGroup -Subscription $AdpSubscription

## Verify Access to Source Server

Before scheduling the export, validate the source SQL Server is accessible.

In [ ]:
$sqlServer = az sql server show --name $LogicalSQLServerName --resource-group $SourceResourceGroup --subscription $AdpSubscription -o JSON | ConvertFrom-JSON
if ($sqlServer)
{
    Write-Host "Source SQL Server: " $sqlServer.name
    if ($sqlServer.state -eq "Ready")
    {
        Write-Host "State: Ready"
    }
    else
    {
        Write-Host "ERROR: Source server is not in Ready state.  Current state is: " $sqlServer.state
    }

    $sqlAzureAdmin = az sql server ad-admin list --server $LogicalSQLServerName --resource-group $SourceResourceGroup --subscription $AdpSubscription -o JSON | ConvertFrom-JSON
    if ($sqlAzureAdmin)
    {
        Write-Host "Azure AD admin set to" $sqlAzureAdmin.login
        Write-Host "SUCCESS: source server accessible and properly configured."
    }
    else
    {
        Write-Host "ERROR: Source server has no Azure AD administrator configured.  The Data Portability solution requires Azure Active Directory to be configured on source servers to provide secure access."
    }

}
else 
{
    Write-Host "ERROR: Source server " $sqlServer.name "not found or current account lacks access to resource."
    Write-Host "Validate input settings:"
    Write-Host "Resource group: " $SourceResourceGroup
    Write-Host "Subscription: " $AdpSubscription
}

## Prepare input variable for Orchestrator Export Function

In [ ]:
$InputForExportFunction = Prepare-InputForExportFunction -Subscription $AdpSubscription -ADPResourceGroupName $AdpResourceGroup `
                             -BatchAccountName $AdpBatch -FunctionName $AdpFunc -VNetName $AdpVNET -SourceRGName $SourceResourceGroup `
                             -SqlServerName $LogicalSQLServerName  -StorageAccountName $StorageAccount
Write-Host "Setting parameter variables for Export Function Call..."
$InputForExportFunction.Header
$InputForExportFunction.FunctionURL
$InputForExportFunction.Body

## Start Export of SQL Server 
Run the cell to start export operation of specified sql server

In [ ]:
$ExportResponse = Invoke-RestMethod -Method 'Post' -Headers $InputForExportFunction.Header -Uri $InputForExportFunction.FunctionURL -Body $InputForExportFunction.Body -ContentType 'application/json'
$ExportResponse

## Get Status of Export Operation
Run the cell to get export operation status

In [ ]:
$statusCheckResponse = Invoke-RestMethod -Method 'Get' -Uri $ExportResponse.statusQueryGetUri


Write-Host "Orchestrator Request: " $statusCheckResponse.name
Write-Host "`tOrchestrator Status: " $statusCheckResponse.runtimeStatus

$outputParams = $statusCheckResponse.output 
if ($outputParams)
{
    $batchJobID = $outputParams.Item2[0]
    $containerUrl = $outputParams.Item2[1]

    Write-Host "`tCreated Export Batch Job ID: " $batchJobId
    Write-Host "`tExport container URL: " $containerUrl

    $azBatchLogin = az batch account login --name $AdpBatch --resource-group $AdpResourceGroup -o JSON | ConvertFrom-Json
    $jobStatus = az batch job show --job-id $batchJobID -o JSON | ConvertFrom-Json
    Write-Host "Export Job running on Pool: " $jobStatus.poolInfo.poolId
    Write-Host "`tExport Request Status: " $jobStatus.state

    $taskList = az batch task list --job-id $batchJobId -o JSON | ConvertFrom-Json
    if ($taskList)
    {
        foreach ($task in $taskList)
        {
            Write-Host "`tDatabase Export Task ID: " $task.id 
            Write-Host "`t`tStatus: " $task.state
            $taskExecution = $task.executionInfo
            if ($taskExecution)
            {
                Write-Host "`t`tResult: " $taskExecution.result
            }
        }
    }
}